In [4]:
import ee
import folium
import geemap.foliumap as geemap
import pandas as pd
import matplotlib.pyplot as plt

# Inicializar la API de Earth Engine
ee.Initialize()

# Definir la región de interés
region = ee.FeatureCollection('projects/ee-mariocaceresreyna25/assets/PROVINCIAS_PERU') \
            .filter(ee.Filter.eq('PROVINCIA', 'HUARAZ'))

# Función para calcular el NDVI
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Lista para almacenar los datos para el gráfico
ndvi_time_series = []

# Procesar imágenes año por año
for year in range(2015, 2024):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    # Cargar la colección de imágenes Landsat 8
    collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                    .filterBounds(region) \
                    .filterDate(start_date, end_date) \
                    .map(addNDVI)
    
    # Calcular el NDVI medio para el año
    mean_ndvi = collection.select('NDVI').mean()
    
    # Agregar el NDVI medio a la lista para el gráfico
    ndvi_value = mean_ndvi.reduceRegion(ee.Reducer.mean(), region, scale=30).get('NDVI').getInfo()
    ndvi_time_series.append(ndvi_value)
    
    # Visualización
    map = geemap.Map()
    map.addLayer(mean_ndvi, {'min': 0, 'max': 1, 'palette': ['blue', 'white', 'green']}, f'NDVI {year}')
    map.centerObject(region, 8)
    display(map)

# Gráfico de la evolución temporal del NDVI
plt.figure(figsize=(10, 6))
plt.plot(range(2015, 2024), ndvi_time_series, marker='o')
plt.title('Evolución Temporal del NDVI (2015-2023)')
plt.xlabel('Año')
plt.ylabel('Valor NDVI medio')
plt.grid(True)
plt.show()



EEException: Your application is authenticating by using local Application Default Credentials. The earthengine.googleapis.com API requires a quota project, which is not set by default. To learn how to set your quota project, see https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds .